# Resampling Methods - Lab

## Introduction

Now that you have some preliminary background on bootstrapping, jacknife and permutation tests, its time to practice those skills by coding them into functions. You'll then apply these tests to a hypothesis test and compare the results to a parametric t-test.

## Objectives

You will be able to:
* Understand permutation testing
* Understand what jacknife is
* Understand what bootstrapping is

## Bootstrapping

Write a function that takes a sample and generates n additional samples of the same size using bootstrapping. (Recall that bootstrapping creates additional sets by sampling with replacement.)

In [1]:
import numpy as np

In [2]:
def bootstrap(sample, n):
    #Your code here
    samples = []
    size = len(sample)
    for i in range(n):
        new_sample = []
        for j in range(size):
            new_sample.append(np.random.choice(sample))
        samples.append(new_sample)
    return samples

In [3]:
stuff = list(range(6))
bootstrap(stuff, 5)

[[4, 0, 5, 5, 2, 4],
 [4, 0, 0, 5, 2, 3],
 [3, 4, 1, 2, 2, 5],
 [3, 3, 0, 0, 0, 2],
 [1, 1, 3, 3, 2, 1]]

## Jacknife 

Write a function that creates additional samples by removing one element at a time. The function should do this for each of the n items in the original sample, returning n samples, each with n-1 members.

In [4]:
def jack1(sample):
    """This function should take in a list of n observations and return n lists
    each with one member (presumably the nth) removed."""
    # Your code here
    samples = []
    for i in range(len(sample)):
        samples.append(sample[:i] + sample[i+1:])
    return samples

In [5]:
jack1(stuff)

[[1, 2, 3, 4, 5],
 [0, 2, 3, 4, 5],
 [0, 1, 3, 4, 5],
 [0, 1, 2, 4, 5],
 [0, 1, 2, 3, 5],
 [0, 1, 2, 3, 4]]

## Permutation Testing

Define a function that generate all possible, equally sized, two set splits of two sets A and B. Sets A and B need not be the same size, but all of the generate two set splits should be of equal size. For example, if we had a set with 5 members and a set with 7 members, the function would return all possible 5-7 splits of the 12 items. 


Here's a more in depth example:  
```A = [1,2,2]
B = [1,3]
permT(A, B) = [
                ([1,2,2], [1,3]),
                ([1,2,3], [1,2]),
                ([1,2,1], [2,3])
                ([1,1,3], [2,2]),
                ([2,2,3], [1,1])
              ]```  
These are all the possible 3-2 member splits of the 5 elements : 1,1,2,2,3.

In [6]:
import itertools
def permT(a,b):
    # Your code here
    conc_list = a + b
    new_as = list(set(itertools.combinations(conc_list, len(a))))
    new_as = [tuple(sorted(i)) for i in new_as]
    new_bs = []
    for new_a in new_as:
        cl_copy = conc_list.copy()
        for a in new_a:
            cl_copy.remove(a)
        new_bs.append(tuple(cl_copy))
    return list(zip(new_as, new_bs))

permT([1, 2, 2], [1, 3])

[((1, 1, 2), (2, 3)),
 ((1, 2, 2), (1, 3)),
 ((1, 1, 3), (2, 2)),
 ((1, 2, 2), (1, 3)),
 ((1, 2, 3), (2, 1)),
 ((2, 2, 3), (1, 1)),
 ((1, 2, 3), (2, 1))]

## Permutation Testing in Practice
Let's further investigate the scenario proposed in the previous lesson. Below are two samples A and B. The samples are mock data for the blood pressure of sample patients. The research study is looking to validate whether there is a statistical difference in the blood pressure of these two groups using a 5% signifincance level.  First, calculate the mean blood pressure of each of the two samples. Then, calculate the difference of these means. From there, use your `permT()` function, defined above, to generate all the possible combinations of the entrie sample data into A-B splits of equivalent sizes as the original sets. For each of these combinations, calculate the mean blood pressure of the two groups and record the difference between these sample means. The full collection of the difference in means between these generated samples will serve as the denominator to calculate the p-value associated with the difference between the original sample means.

For example, in our small handwritten example above:

$\mu_a = \frac{1+2+2}{3} = \frac{5}{3}$  
and  
$\mu_b = \frac{1+3}{2} = \frac{4}{2} = 2$  

Giving us

$\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$

In comparison, for our various combinations we have:

([1,2,2], [1,3]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$  
([1,2,3], [1,2]):  $\mu_a - \mu_b = 2 - \frac{3}{2} = \frac{1}{2}$  
([1,2,1], [2,3]):  $\mu_a - \mu_b = \frac{4}{3} - \frac{5}{3} = -\frac{1}{2}$  
([1,1,3], [2,2]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$  
([2,2,3], [1,1]):  $\mu_a - \mu_b = \frac{7}{3} - 1 = \frac{4}{3}$  

A standard hypothesis test for this scenario might be:

$h_0: \mu_a = \mu_b$  
$h_1: \mu_a < \mu_b$  
  
Thus comparing our sample difference to the differences of our possible combinations, we look at the number of experiments from our permutation space that were the same or greater then our sample statistic, divided by the total number of permutations. In this case, 4 out of 5 of the permutation cases produced the same or greater differences in the two sample means. This value .8 is a strong indication that we cannot refute the null hypothesis for this instance.



In [7]:
a = [109.6927759 , 120.27296943, 103.54012038, 114.16555857,
       122.93336175, 110.9271756 , 114.77443758, 116.34159338,
       112.66413025, 118.30562665, 132.31196515, 117.99000948]
b = [123.98967482, 141.11969004, 117.00293412, 121.6419775 ,
       123.2703033 , 123.76944385, 105.95249634, 114.87114479,
       130.6878082 , 140.60768727, 121.95433026, 123.11996767,
       129.93260914, 121.01049611]

In [11]:
# Your code here
import numpy as np
mu_a, mu_b = np.mean(a), np.mean(b)
dif_ab = mu_a - mu_b
perm_ab = permT(a, b)
perm_ab

[((103.54012038,
   109.6927759,
   110.9271756,
   114.16555857,
   114.87114479,
   118.30562665,
   121.6419775,
   122.93336175,
   123.2703033,
   123.76944385,
   130.6878082,
   140.60768727),
  (120.27296943,
   114.77443758,
   116.34159338,
   112.66413025,
   132.31196515,
   117.99000948,
   123.98967482,
   141.11969004,
   117.00293412,
   105.95249634,
   121.95433026,
   123.11996767,
   129.93260914,
   121.01049611)),
 ((105.95249634,
   109.6927759,
   110.9271756,
   114.87114479,
   120.27296943,
   121.01049611,
   121.95433026,
   123.11996767,
   123.2703033,
   130.6878082,
   132.31196515,
   140.60768727),
  (103.54012038,
   114.16555857,
   122.93336175,
   114.77443758,
   116.34159338,
   112.66413025,
   118.30562665,
   117.99000948,
   123.98967482,
   141.11969004,
   117.00293412,
   121.6419775,
   123.76944385,
   129.93260914)),
 ((103.54012038,
   105.95249634,
   109.6927759,
   114.16555857,
   114.77443758,
   120.27296943,
   121.01049611,
  

In [21]:
means = []
for comb in perm_ab[:]:
    mu_one, mu_two = np.mean(comb[0]), np.mean(comb[1])
    means.append(mu_one - mu_two)
means

[-1.7825349559523858,
 1.3538908938095489,
 -2.5340895367857286,
 -1.1652890332143073,
 -0.40611861440474684,
 -5.646905120833338,
 1.130294968690464,
 -3.4291058900000024,
 4.785345082619045,
 -0.33019010404761673,
 -3.2833858690476347,
 2.903607172619033,
 -7.776579115952401,
 8.180151102976211,
 4.84940366595238,
 -0.6634545274999937,
 -1.7142888961904674,
 0.8479219795238038,
 2.759966959761897,
 -1.9669428654762129,
 -3.4555160167857224,
 0.21694808619049866,
 -2.470762971071423,
 -3.4734192654761955,
 3.275490792023831,
 3.7984443920238107,
 2.2842817698809483,
 0.8364103046428539,
 -3.9120265796428555,
 -6.718334300000009,
 3.501392302738097,
 2.8409631425000157,
 3.2188040722619036,
 -3.651119362261923,
 0.49208758285712406,
 2.360935480595245,
 0.029651613809534183,
 -3.5954720884523965,
 0.8839233221428344,
 2.187058544523822,
 2.201330305119029,
 -1.6114095070238363,
 3.1017179101190777,
 4.861304344166655,
 2.9101658041666667,
 -0.5327830790476469,
 3.373580380714259,
 2.38

In [23]:
num = 0
den = len(means)
for mean in means:
    num += 1 if mean >= dif_ab else 0
num/den

0.9890762811021258

## T-test Revisited

The parameteric statistical test equivalent to our permutation test above would be a t-test of the two groups. Perform a t-test on the same data above in order to calculate the p-value. How does this compare to the above results?

In [24]:
# Your code here
import scipy.stats as stats
num = np.mean(a) - np.mean(b)
s = np.var(a+b)
n = len(a+b)
denom = s/np.sqrt(n)
t = num / denom
pval = stats.t.sf(np.abs(t), n-1)*2
print(pval)

0.6196331755824978


## Bootstrap Applied

Use your code above to apply the bootstrap technique to this hypothesis testing scenario. In other words, similar to the permutation testing you performed above, compute additional samples (arbitrarily let's say 1000) of the same size as the original sample, with replacement. For each of these additional samples, compute whether the difference in sample means is the same or greater then that of the original samples. Use this to calculate an overall p-value for the null hypothesis.

In [27]:
# Your code here
mean_diffs = []
a_boot = bootstrap(a, 10000)
b_boot = bootstrap(b, 10000)
for i in range(10000):
    mean_diffs.append(np.mean(a_boot[i]) - np.mean(b_boot[i]))
count = 0
for diff in mean_diffs:
    count += 1 if diff >= dif_ab else 0
count/len(mean_diffs)

0.5026

## Summary

Well done! In this lab you practice coding modern statistical resampling techniques of the 20th century! You also started to compare these non-parametric methods to other parametric methods such as the t-test that we previously discussed.